## Analyse the Extracted Samples

In [1]:
# import necessary modules
#*****************
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import warnings
import tqdm
import pandas as pd
warnings.filterwarnings("ignore")

%matplotlib inline
import matplotlib as mlp
mlp.rc("xtick",labelsize=12)
mlp.rc("ytick",labelsize=12)
mlp.rc("axes",labelsize=14)
plt.rcParams["figure.figsize"] = [10,5]
plt.rcParams["figure.dpi"] = 100

CURR_DIR = os.getcwd()

In [2]:
print(CURR_DIR)

/home/upendra/Documents/iot_project/Silent-Interface-for-IOT-Devices/codes


In [3]:
# make necessary definitions

MAIN_DIR = os.path.join(CURR_DIR,"..")
DATA_DIR = os.path.join(MAIN_DIR,"dataset")
FIG_DIR = os.path.join(MAIN_DIR,"figures")
os.makedirs(FIG_DIR,exist_ok=True)

SPEAKER = ["RL","RN","SR","US"]
SESSION = ["session1","session2","session3"]
MODE = ["mentally","mouthed","audible"]
SENTENCES =["अबको समय सुनाउ","एउटा सङ्गित बजाउ","आजको मौसम बताउ","बत्तिको अवस्था बदल","पङ्खाको स्तिथी बदल"]
WORDS = ["समय","सङ्गित","मौसम","बत्ति","पङ्खा"]

SENTENCE_LABEL=[0,1,2,0,3,1,0,3,0,0,1,1,3,3,4,4,2,3,1,2,2,2,4,4,4]
WORD_LABEL=[4,0,3,1,0,1,1,0,4,0,3,2,4,4,2,1,4,1,2,2,2,0,3,3,3]

SAMPLING_RATE = 250

# a function to save plotted figures
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(FIG_DIR, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    

In [4]:
print(MAIN_DIR)
print(DATA_DIR)
print(len(set(SENTENCE_LABEL)))
print(len(set(WORD_LABEL)))

/home/upendra/Documents/iot_project/Silent-Interface-for-IOT-Devices/codes/..
/home/upendra/Documents/iot_project/Silent-Interface-for-IOT-Devices/codes/../dataset
5
5


In [5]:
all_files = glob.glob(DATA_DIR+"/*/*/*/*/*.txt",recursive=True)
print(len(all_files))

90


In [6]:
def parser(files):
    """
    parser function to extract utterances from .txt file and store them in a dictionary
    """
    dataset = {"data":[], "speaker":[],"session":[], "mode":[], "utter_type":[],"file_name":[]}
    def get_data(file):
        file_name = file.split("/")[-1]
        mode = file.split("/")[-2]
        utter_type = file.split("/")[-3]
        session = file.split("/")[-4]
        speaker = file.split("/")[-5]
        f = open(file, 'r')
        contents = map(lambda x : x.strip(), f.readlines())
        #the file starts with '%' and some instruction before data and removing these data 
        frames_original = list(filter(lambda x : x and x[0] != '%', contents))[1:]
        print("frames_original:", frames_original)
        #the data row contains channels info digital trigger and accelerometer info separated by comma
        frames_original = list(map(lambda s : list(map( lambda ss: ss.strip(), s.split(','))), frames_original))
        # (8 channels) + digital triggers
        # the digital trigger is in a[16], used to indicate the utterance
        frames = list(map(lambda a: list(map(float, a[1:9])) + [float(a[16])] , frames_original))
        frames = np.array(frames)
        indices = []
        signal = []
        for index,f in enumerate(frames[:,-1]):
            if(bool(f) ^ bool(frames[(index+1) if ((index+1)<len(frames)) else index,-1]) ):
                indices.append(index)
                if len(indices)>1 and len(indices)%2==0:
                    signal.append(frames[indices[len(indices)-2]:indices[len(indices)-1],:-1])   
        
        dataset["data"].extend(signal)
        dataset["speaker"].extend([speaker]*len(signal))
        dataset["session"].extend([session]*len(signal))
        dataset["mode"].extend([mode]*len(signal))
        dataset["utter_type"].extend([utter_type]*len(signal))
        dataset["file_name"].extend([file_name]*len(signal))
        
    for file,i in zip(files,tqdm.tqdm(range(1,len(files)+1),desc="PARSING DATA")):
        get_data(file)
    
    return dataset

In [7]:
all_data = parser(all_files)

PARSING DATA:   0%|          | 0/90 [00:00<?, ?it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

PARSING DATA:   1%|          | 1/90 [00:02<03:24,  2.30s/it]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

PARSING DATA:   2%|▏         | 2/90 [00:03<02:46,  1.89s/it]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variabl

PARSING DATA:  23%|██▎       | 21/90 [00:39<02:22,  2.07s/it]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

PARSING DATA:  24%|██▍       | 22/90 [00:41<02:31,  2.23s/it]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

PARSING DATA:  26%|██▌       | 23/90 [00:45<02:51,  2.56s/it]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the con

PARSING DATA:  47%|████▋     | 42/90 [01:17<01:00,  1.26s/it]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

PARSING DATA:  48%|████▊     | 43/90 [01:18<01:07,  1.43s/it]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

PARSING DATA:  49%|████▉     | 44/90 [01:20<01:13,  1.59s/it]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the con

PARSING DATA:  70%|███████   | 63/90 [01:53<00:46,  1.73s/it]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

PARSING DATA:  71%|███████   | 64/90 [01:56<00:50,  1.94s/it]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

PARSING DATA:  72%|███████▏  | 65/90 [01:57<00:41,  1.65s/it]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the con

PARSING DATA:  93%|█████████▎| 84/90 [02:28<00:10,  1.79s/it]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

PARSING DATA:  94%|█████████▍| 85/90 [02:33<00:13,  2.74s/it]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

PARSING DATA:  96%|█████████▌| 86/90 [02:35<00:10,  2.63s/it]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the con

In [8]:
#all_data["data"][0].shape
all_data["data"][1].shape
all_data["data"][1]

array([[-11443.28808594,  21903.07617188,  38173.515625  , ...,
         -4462.48095703,  18627.85351562,  14327.6015625 ],
       [-11436.18066406,  21902.60742188,  38169.6484375 , ...,
         -4465.58789062,  18629.55273438,  14318.21386719],
       [-11462.33203125,  21866.82226562,  38135.046875  , ...,
         -4485.92773438,  18600.16015625,  14325.34472656],
       ...,
       [-11377.93164062,  21921.62890625,  38117.078125  , ...,
         -4588.14257812,  18559.83789062,  14216.53613281],
       [-11404.95507812,  21881.10546875,  38064.57421875, ...,
         -4627.45898438,  18519.6484375 ,  14193.93847656],
       [-11419.77441406,  21852.13671875,  38046.265625  , ...,
         -4615.21044922,  18511.53515625,  14203.7734375 ]])

In [9]:
df = pd.DataFrame(all_data)

In [10]:
df

,data,speaker,session,mode,utter_type,file_name
0,"[[-11383.8994140625, 22032.203125, 38198.10156...",US,session1,audible,words,OpenBCI-RAW-2021-05-13_19-31-31.txt
1,"[[-11443.2880859375, 21903.076171875, 38173.51...",US,session1,audible,words,OpenBCI-RAW-2021-05-13_19-31-31.txt
2,"[[-11539.0654296875, 21800.974609375, 38084.98...",US,session1,audible,words,OpenBCI-RAW-2021-05-13_19-31-31.txt
3,"[[-11561.328125, 21838.123046875, 38113.367187...",US,session1,audible,words,OpenBCI-RAW-2021-05-13_19-31-31.txt
4,"[[-11538.037109375, 21841.69921875, 38132.9453...",US,session1,audible,words,OpenBCI-RAW-2021-05-13_19-31-31.txt
...,...,...,...,...,...,...
2268,"[[-50409.4609375, 26914.673828125, 535.3466186...",US,session3,mentally,sentences,OpenBCI-RAW-2021-05-15_11-01-11.txt
2269,"[[-50491.44921875, 26991.720703125, 545.516662...",US,session3,mentally,sentences,OpenBCI-RAW-2021-05-15_11-01-11.txt
2270,"[[-50506.04296875, 27109.8046875, 654.95080566...",US,session3,mentally,sentences,OpenBCI-RAW-2021-05-15_11-01-11.txt
2271,"[[-50427.56640625, 26956.068359375, 519.163940...",US,session3,mentally,sentences,OpenBCI-RAW-2021-05-15_11-01-11.txt


In [11]:
#df.head(10)



In [12]:
all_file_name = df["file_name"]
print(len(all_file_name))

2273


In [13]:
all_words = df[df["utter_type"]=="words"]
all_sentences = df[df["utter_type"]=="sentences"]
all_word_length = [x.shape[0] for x in all_words["data"]]
all_sent_length = [x.shape[0] for x in all_sentences["data"]]

all_words_audible = df.query('utter_type=="words" and mode=="audible"')
all_sent_audible = df.query('utter_type=="sentences" and mode=="audible"')


print(len(all_words))
print(len(all_sentences))
#all_sentences

1134
1139


In [48]:
total_utter = len(all_files)*25   # 125 utterances per file
sent_utter = word_utter = total_utter*0.5
# print("error word recordings: ", all_words-word_utter)
# print("error sentence recordings: ", all_sentences-sent_utter)


def error_files(X,Y,sec): # sec:minimun uttrance time: 1 for word and 2.5 for sentence
    #err_length = []
    err_position = []
    err_file_name = []
    count = 0
    
    for word in X:
        if (word < (sec*250)):       
            #err_length.append(word)
            err_position.append(count)
        count+=1
   # print(len(err_word_length))
   # print(len(err_word_position))
   
    for i in err_position:
        err_file_name.append([Y[i]])

    return(np.unique((err_file_name)),len(np.unique(err_file_name)))

if (len(all_sentences)>sent_utter):    
    print("error sentence file names:",error_files(all_sent_length,all_file_name,2.5))
# checking files for error word

if (len(all_words)>word_utter):    
    print("error word file names:",error_files(all_word_length,all_file_name,1.58))
    
    
len(all_files)

error sentence file names: (array(['OpenBCI-RAW-2021-05-13_17-39-07.txt',
       'OpenBCI-RAW-2021-05-13_17-43-48.txt',
       'OpenBCI-RAW-2021-05-13_17-47-24.txt',
       'OpenBCI-RAW-2021-05-13_17-55-06.txt',
       'OpenBCI-RAW-2021-05-13_18-15-06.txt',
       'OpenBCI-RAW-2021-05-13_18-19-13.txt',
       'OpenBCI-RAW-2021-05-13_18-42-41.txt',
       'OpenBCI-RAW-2021-05-13_19-12-44.txt',
       'OpenBCI-RAW-2021-05-13_19-16-02.txt',
       'OpenBCI-RAW-2021-05-13_19-26-37.txt',
       'OpenBCI-RAW-2021-05-13_19-33-51.txt',
       'OpenBCI-RAW-2021-05-13_19-36-21.txt',
       'OpenBCI-RAW-2021-05-14_11-08-20.txt',
       'OpenBCI-RAW-2021-05-14_11-45-22.txt',
       'OpenBCI-RAW-2021-05-14_12-16-59.txt'], dtype='<U35'), 15)
error word file names: (array(['OpenBCI-RAW-2021-05-13_17-39-07.txt',
       'OpenBCI-RAW-2021-05-13_18-00-51.txt',
       'OpenBCI-RAW-2021-05-13_18-15-06.txt',
       'OpenBCI-RAW-2021-05-13_18-32-20.txt',
       'OpenBCI-RAW-2021-05-13_18-45-18.txt',
       '

90

In [ ]:
all_sent_audible

In [ ]:
all_words_audible

In [ ]:
plt.hist(all_word_length,alpha=0.4)
plt.hist(all_sent_length,alpha=0.5)
plt.xlabel("Sample Length")
plt.ylabel("Samples")
# save_fig("length distribution")
plt.show()

In [ ]:
print("Max length of words utterances in seconds: ",np.max(all_word_length)/SAMPLING_RATE)
print("Max length of sentence utterances in seconds: ",np.max(all_sent_length)/SAMPLING_RATE)
print("Min length of words utterances in seconds: ",np.min(all_word_length)/SAMPLING_RATE)
print("Min length of sentence utterances in seconds: ",np.min(all_sent_length)/SAMPLING_RATE)

In [ ]:
sample = np.transpose(all_data["data"][1])
print(all_data["speaker"][1])
print(all_data["mode"][1])
print(sample.shape)

sample2 = np.transpose(all_words_audible["data"].iloc[20])
print(all_words_audible["speaker"].iloc[20])
print(all_words_audible["mode"].iloc[20])
print(sample2.shape)

In [ ]:
fig, axes = plt.subplots(nrows=8,ncols=1)
for i in range(8):
#     axes[i].plot(sample[i])
    axes[i].plot(sample2[i])
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=8,ncols=1)
for i in range(8):
    axes[i].plot(sample[i])
#     axes[i].plot(sample2[i])
plt.show()

In [ ]:
from scipy.fft import fft,fftfreq
from scipy.fftpack import fft
from scipy.stats import zscore
sample = np.transpose(sample)
N = len(sample[:,0])
y = fft(zscore(sample[:,0]))
x = fftfreq(N,1/250)

sample2 = np.transpose(sample2)
N = len(sample2[:,0])
Y = fft(zscore(sample2[:,0]))
X = fftfreq(N,1/250)

plt.plot(abs(x),abs(y))
plt.plot(abs(X),abs(Y))
plt.legend(["mentally","audible"])
plt.show()